# Защита персональных данных клиентов

Необходимо разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию.

Необходимо защитить данные так, чтобы при преобразовании качество моделей машинного обучения не ухудшилось.

### Цель

Разработать метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию.

### Задачи

1. Изучить данные
2. Ответить на поставленный вопрос
3. Предложить алгорим преобразования данных
4. Запрограммировать этот алгоритм
5. Выполнить проверку


### Описание данных

* Признаки: пол, возраст и зарплата застрахованного, количество членов его семьи.
* Целевой признак: количество страховых выплат клиенту за последние 5 лет.

## Загрузка и подготовка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [2]:
try:
    df = pd.read_csv('insurance.csv')
except:
    df = pd.read_csv('/datasets/insurance.csv')

Изучим данные

In [3]:
df.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [5]:
df['Страховые выплаты'].unique()

array([0, 1, 2, 3, 5, 4])

In [6]:
df['Пол'].unique()

array([1, 0])

In [7]:
df['Члены семьи'].unique()

array([1, 0, 2, 4, 3, 5, 6])

In [8]:
df['Возраст'].unique()

array([41., 46., 29., 21., 28., 43., 39., 25., 36., 32., 38., 23., 40.,
       34., 26., 42., 27., 33., 47., 30., 19., 31., 22., 20., 24., 18.,
       37., 48., 45., 44., 52., 49., 35., 56., 65., 55., 57., 54., 50.,
       53., 51., 58., 59., 60., 61., 62.])

Пропусков в данных нет. Количественные признаки: 'Возраст', 'Зарплата'. Категориальные признаки: "Пол", "Члены семьи". Целевой признак "Страховые выплаты" является количественным, т.к по условию требуется применение линейной регрессии.

Разделим данные на обучающие и целевой признаки

In [9]:
features = df.drop('Страховые выплаты', axis=1)

In [10]:
target = df['Страховые выплаты']

Разобьем данные на обучающую и валидационную выборки

In [11]:
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.25, random_state=12345)

## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** если умножить признаки на обратимую матрицу, качество линейной регрессии не изменится.

**Обоснование:**

Если подставить в формулу для предсказаний преобразованные признаки обучающей выборки и преобразованную формулу обучения, получим:

$$
a = (X A) w' = X A A^{-1} w = X E w = X w
$$

Получаем исходную формулу. Значит, домножение признаков на обратимую матрицу не повлечет за собой изменения работы линейной регрессии.

## Алгоритм преобразования

**Алгоритм**


Используя генератор, получим случайную обратимую матрицу $A$, размером $n \times n$, где n — чиcло столбцов в матрице признаков $X$.

Затем преобразуем исходные призаки путем скалярного умножения матрицы $X$ на матрицу $A$.

Для декодирования признаков будем использовать обратную матрицу $A^{-1}$.

**Обоснование**

По свойству обратной матрицы :

$$
X A A^{-1} = X E = X,
$$

что позволит легко декодировать зашифрованные данные. Также, как было доказано выше, умножение матрицы на случайную обратимую матрицу не влияет на качество линейной регрессии.

## Проверка алгоритма

**Используем модель линейной регрессии без преобразования данных**

In [12]:
model = LinearRegression()

In [13]:
model.fit(features_train, target_train)

LinearRegression()

In [14]:
predictions = model.predict(features_test)

Оценим качество модели

In [15]:
r2_score(target_test, predictions)

0.43522757127026546

**Преобразуем данные по выбранному алгоритму**

Создадим функцию для преобразования данных

In [16]:
def encode(feat_train, feat_test):
    n = feat_train.shape[1] # число столбцов исходной матрицы
    while True: # нужно случайным образом получит обратимую матрицу
        A = np.random.normal(0, 1, (n, n)) # случайная матрица
        try: # проверка на обратимость
            np.linalg.inv(A)
        except:
            continue
        break
    return A, feat_train @ A, feat_test @ A




Преобразуем данные

In [17]:
A, new_features_train, new_features_test = encode(features_train, features_test)


Создадим модель

In [18]:
model = LinearRegression()

In [19]:
model.fit(new_features_train, target_train)

LinearRegression()

In [21]:
predictions = model.predict(new_features_test)

Оценим качество модели

In [22]:
r2_score(target_test, predictions)

0.4352275712703151

Получили то же значение

Декодируем данные

In [23]:
new_features_train @ np.linalg.inv(A)

,0,1,2,3
3369,1.000000e+00,43.0,36200.0,1.000000e+00
1441,1.000000e+00,34.0,57600.0,6.680457e-12
571,6.092956e-13,32.0,41100.0,1.000000e+00
225,-1.226806e-12,36.0,45100.0,1.000000e+00
2558,1.544397e-14,33.0,50600.0,2.000000e+00
...,...,...,...,...
3497,1.842532e-14,42.0,32100.0,-3.993255e-12
3492,-4.655415e-14,28.0,22700.0,4.000000e+00
2177,1.000000e+00,41.0,44700.0,1.000000e+00
3557,5.366176e-13,22.0,50100.0,4.000000e+00


In [24]:
features_train

,Пол,Возраст,Зарплата,Члены семьи
3369,1,43.0,36200.0,1
1441,1,34.0,57600.0,0
571,0,32.0,41100.0,1
225,0,36.0,45100.0,1
2558,0,33.0,50600.0,2
...,...,...,...,...
3497,0,42.0,32100.0,0
3492,0,28.0,22700.0,4
2177,1,41.0,44700.0,1
3557,0,22.0,50100.0,4


Как видим, даннные после декодирования практически не изменились

# Вывод

Итак, в качестве метода по защите данных клиентов было предложено умножение матрицы на случайную обратимую матрицу. Было доказано, что данная операция не повлияет на качество модели линейной регрессии. Декодирование возможно путем домножения преобразованной матрицы на матрицу, обратную к использованной случайной.